In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

train_X = pd.read_csv('../data/trn_X_20231211_final_col_01_transformed.csv')
train_y = pd.read_csv('../data/trn_y_20231211_final_col_01_transformed.csv')

In [2]:
train_X.isnull().sum()

age            0
ainc           0
HE_sbp         0
HE_dbp         0
HE_ht          0
              ..
HE_Ubil_0.0    0
HE_Ubil_4.0    0
DI3_pt_0.0     0
DI3_pt_1.0     0
DI3_pt_8.0     0
Length: 208, dtype: int64

In [3]:
### 임시 -transform
def transform(X, y, num_features, onehot_features, label_features, y_related):
    ### 결측값 처리 3 - fillna: KNN
    from sklearn.impute import KNNImputer
    from sklearn.impute import SimpleImputer

    # 1) num_features 결측값 처리 - KNN
    knn_imp = KNNImputer(n_neighbors=5)
    X[num_features] = knn_imp.fit_transform(X[num_features])

    # 2) onehot_features, label_features 결측값 처리 - 최빈값(most frequent values)
    freq_imp = SimpleImputer(strategy = "most_frequent")
    X[onehot_features] = freq_imp.fit_transform(X[onehot_features])
    X[label_features] = freq_imp.fit_transform(X[label_features])

    # transform array(X) to dataframe
    df_new = pd.DataFrame(X, columns=X.columns) 
    df_new.isna().sum().sort_values(ascending = False)

    ### 인코딩
    # 1. numeric # 1. numeric - scaling 
    from sklearn.preprocessing import MinMaxScaler 
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_new[num_features] = scaler.fit_transform(df_new[num_features])
    # 2. onehot -> onehot 인코딩으로 진행
    df_new = pd.get_dummies(df_new, columns=onehot_features) # sklearn OneHotEncoder 사용도 가능함

    ### 샘플링
    # under sampling.  OVER 만 하기엔 너무 많이 생성해야해서 under 선수행
    from imblearn.under_sampling import RandomUnderSampler

    X = df_new
    rus = RandomUnderSampler(sampling_strategy=0.5, random_state=42) 
    X_under, y_under = rus.fit_resample(X, y)
    X_under.shape, y_under.shape

    # over sampling
    from imblearn.over_sampling import SMOTE

    smote = SMOTE(random_state=42)
    X_over, y_over = smote.fit_resample(X_under, y_under)

    X_over.shape, y_over.shape

    return X_over, y_over


In [5]:
# numerical = ['age', 'HE_wc', 'ainc', 'HE_ht', 'HE_wt']
# onehot = ['LQ1_sb', 'BO1', 'DI1_pt', 'DE1_pr', 'DI2_pr', 'BH2_61', 'DI3_pr', 'BO1_1', 'marri_1', 'DI2_pt', 'BD1', 'EC_stt_2', 'DI3_pt', 'live_t', 'npins', 'EC_wht_0', 'LQ2_ab', 'BH1', 'D_2_1', 'DJ4_pr', 'DJ4_pt', 'DI1_pr', 'LQ4_00', 'DE1_pt', 'educ', 'EC_occp', 'sex']
# label = ['BE5_1', 'BE3_31', 'D_1_1', 'cfam', 'ho_incm', 'BP1', 'incm5', 'edu', 'incm', 'DI3_2', 'ho_incm5']

# y_related = ['BP_PHQ_1', 'BP_PHQ_2', 'BP_PHQ_3', 'BP_PHQ_4', 'BP_PHQ_5', 'BP_PHQ_6', 'BP_PHQ_7', 'BP_PHQ_8', 'BP_PHQ_9', 'mh_PHQ_S', 'BP6_10', 'BP6_31', 'DF2_pr', 'DF2_pt', 'BP1']

In [6]:
# X, y = transform(train_X, train_y, numerical, onehot, label, y_related)

In [7]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# # 각 장르에 대해 VIF 계산 함수 정의
# def calculate_vif(data, variables):
#     vif_data = data[variables] 
#     vif = pd.DataFrame()
#     vif["Variable"] = vif_data.columns
#     vif["VIF"] = [variance_inflation_factor(vif_data.values, i) for i in range(vif_data.shape[1])]
#     return vif

# calculate_vif(X, X.columns)

In [9]:
train_y

,depressed
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
26847,1.0
26848,1.0
26849,1.0
26850,1.0


In [10]:
train_y.value_counts()

depressed
0.0          13426
1.0          13426
Name: count, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y['depressed'])

# Logistic Regression 모델 생성 및 학습
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


# y를 많이 뻥튀기 할 수록 정확도는 올라감!!!

/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:605: Fu

              precision    recall  f1-score   support

         0.0       0.75      0.87      0.81      2686
         1.0       0.85      0.71      0.77      2685

    accuracy                           0.79      5371
   macro avg       0.80      0.79      0.79      5371
weighted avg       0.80      0.79      0.79      5371



/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is d